In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import time
#import skfda
os.getcwd()

'C:\\Users\\CristhianJohnatanIzq\\OneDrive - SQDM\\Proyecto2\\ENVIO_DARIO\\ANALISIS_VELOCIDADES_NODOS_SEGMENTADOS_OOKLA'

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

# Leyendo archivos de nodos segmentados

Los archivos de nodos segmentados desde mayo deben tener nombres con la siguiente estructura:

Nodos_Segmentados_Mayo_21.csv   (EL NOMBRE DEL MES DEBE ESTAR COMPLETO VER dic_meses_seg EN EL SIGUIENTE BLOQUE DE CODIGO

Si el archivo remitido por Jorge Saenz tiene extensión xlsx convertirlo a csv

In [3]:
path_file = 'DATA_OOKLA/archivos_nodos_segmentados_2021/'
dic_meses_seg = {'ENERO':202101,'FEBRERO':202102,'MARZO':202103,'ABRIL':202104, 'BACK MARZO':202104,'MAYO':202105,
                'JUNIO':202106, 'JULIO':202107, 'AGOSTO':202108, 'SEPTIEMBRE':202109, 'OCTUBRE':202110, 
                 'NOVIEMBRE':202111, 'DICIEMBRE':202112}


df_segmentados_abril = pd.read_csv(f'{path_file}Segmentaciones_a_Abril_21.csv', sep=';', decimal=',')
df_segmentados_abril = df_segmentados_abril.explode('Nodo')
df_segmentados_abril = df_segmentados_abril[['Nodo','CRONOGRAMA VIGENTE']].copy()
df_segmentados_abril.columns = ['Nodo','mes']

# se agrega el nodo ERB que se había considerado en marzo pero no aparecía en el archivo hasta abril
df_segmentados_abril = df_segmentados_abril.append(pd.DataFrame.from_dict(data={'Nodo':['ERB'],'mes':['MARZO']}),
                                                   ignore_index=True)
df_segmentados_abril['mes'] = [dic_meses_seg[x] for x in df_segmentados_abril['mes']]
df_segmentados_abril = df_segmentados_abril[['Nodo','mes']]

list_df = [df_segmentados_abril]

path_file = path_file + 'desde_mayo/'
archivos_segmentados_desde_mayo = os.listdir(path_file)



for file in archivos_segmentados_desde_mayo:
    df_segmentados_mes = pd.read_csv(path_file + file, sep=';', decimal=',')
    df_segmentados_mes['Nodo'] = df_segmentados_mes['Nodo'].str.split(',')
    df_segmentados_mes = df_segmentados_mes.explode('Nodo')
    df_segmentados_mes = df_segmentados_mes[['Nodo']]
    mes = file[18:-7]
    print(mes)
    df_segmentados_mes['mes'] = dic_meses_seg[mes.upper()]
    list_df.append(df_segmentados_mes)
    
df_segmentados = pd.concat(list_df)
df_segmentados['mes'].value_counts()

Junio
Mayo


202106    206
202103     87
202105     85
202102     70
202104     46
202101     39
Name: mes, dtype: int64

# OOKLA

Archivo de OOKLA_FIJA_202106 se utiliza las columnas de la primera línea del siguien bloque de código, se debe cambiar el nombre del archivo al actualizarlo, el archivo actualizado debe tener la misma estructura.

In [4]:
t = time.time()
cols = ["FLAG_LAT","OPERADOR", "DOWNLOAD_KBPS","UPLOAD_KBPS", "NODO","FECHA"]
df_ookla = pd.read_csv('DATA_OOKLA/OOKLA_FIJA.csv', sep=',', usecols=cols)
# seleecionando sólo info de los nodos segmentados
nodos_seg = df_segmentados['Nodo'].values
mask_seg = [x in nodos_seg for x in df_ookla['NODO']] & (df_ookla['FLAG_LAT']==0)
df_ookla = df_ookla[mask_seg].copy()
print(f'Tiempo de ejecución en minutos: {(time.time()-t)/60}')

Tiempo de ejecución en minutos: 2.1409747838974


In [5]:
df_segmentados.rename(columns={'Nodo':'NODO','mes':'MES_SEGMENTADO'}, inplace=True)
df_ookla['FECHA'] = [int(x.replace('-','')[:-2]) for x in df_ookla['FECHA']]
df_ookla = df_ookla.merge(df_segmentados, on='NODO', how='right')
df_ookla.info(show_counts =True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168589 entries, 0 to 168588
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   FLAG_LAT        168579 non-null  float64
 1   OPERADOR        168579 non-null  object 
 2   DOWNLOAD_KBPS   168579 non-null  float64
 3   UPLOAD_KBPS     168579 non-null  float64
 4   NODO            168589 non-null  object 
 5   FECHA           168579 non-null  float64
 6   MES_SEGMENTADO  168589 non-null  int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 10.3+ MB


In [6]:
df_ookla['SEGMENTADO'] = (df_ookla['FECHA'] > df_ookla['MES_SEGMENTADO'])
df_ookla['CLARO'] = df_ookla['OPERADOR']=='CLARO'
df_ookla['SEGMENTADO'] = df_ookla['SEGMENTADO'] & df_ookla['CLARO']

In [7]:
df_ookla.to_csv('DATA_OOKLA/salidas/ookla_nodos_segmentados_operadores.csv', sep=';', decimal=',', index=False)

## Tablas CSV OPERADORES Y CLARO PRE Y POS SEGMENTACIÓN

***Revisar medida de centraliad para la agregación***

Se toma como medida de agregación la trimedia definida en las siguientes dos líneas de código. Se puede considerar medidas tradicionales como media o mediana pero se recomienda dejar la trimedia ya que tiene información de la mediana y de los percentiles 10 y 90.

In [8]:
fun_tri = lambda array: (np.percentile(array,10) + 2*np.percentile(array,50) + np.percentile(array,90))/4
fun_tri_3 = lambda array: fun_tri(array) if len(array)>3 else np.nan

table_down = pd.pivot_table(data=df_ookla, columns=['OPERADOR','SEGMENTADO'], index=['NODO'],
               values='DOWNLOAD_KBPS', aggfunc=fun_tri_3)
table_down.columns = [f'{x[0]} SEG {x[1]} TRIDOWN' for x in table_down.columns]

list_pre_cols = [x[:-4] for x in table_down.columns]

table_up = pd.pivot_table(data=df_ookla, columns=['OPERADOR','SEGMENTADO'], index=['NODO'],
               values='UPLOAD_KBPS', aggfunc=fun_tri_3)
table_up.columns = [f'{x[0]} SEG {x[1]} TRIUP' for x in table_up.columns]

table = table_down.join(table_up)
list_scores = []
for col in list_pre_cols:
    col_score = f'SPEED SCORE {col[:-4]}'
    list_scores.append(col_score)
    table[col_score] = 0.9*table[f'{col}DOWN'] + 0.1*table[f'{col}UP']
table = table[list_scores].copy()

In [9]:
table.to_csv('DATA_OOKLA/salidas/ookla_score_nodos_segmentados.csv', sep=';', decimal=',')

In [10]:
table.describe().to_csv('DATA_OOKLA/salidas/ookla_score_nodos_segmentados_resumen.csv', sep=';', decimal=',')

# CSV PARA DASHBOARDS COMPORTAMIENTOS POR MES

In [11]:
df_down_group = df_ookla.groupby(['NODO','FECHA','OPERADOR','SEGMENTADO','MES_SEGMENTADO']).agg({'DOWNLOAD_KBPS':'mean'})
df_up_group = df_ookla.groupby(['NODO','FECHA','OPERADOR','SEGMENTADO','MES_SEGMENTADO']).agg({'UPLOAD_KBPS':'mean'})
df_group = df_down_group.merge(df_up_group, how='inner', on = ['NODO','FECHA','OPERADOR','SEGMENTADO','MES_SEGMENTADO'])
df_group.reset_index(inplace=True)
df_group['FECHA'] = pd.to_datetime([f'{str(x)[:4]}/{str(x)[4:6]}/01' for x in df_group['FECHA']])
df_group.loc[((df_group['SEGMENTADO']) & (df_group['OPERADOR']=='CLARO')),'OPERADOR'] = 'CLARO_POS_SEGMENTACION'
df_group.loc[((~df_group['SEGMENTADO'])& (df_group['OPERADOR']=='CLARO')),'OPERADOR'] = 'CLARO_PRE_SEGMENTACION'

## Velocidad promedio ofrecida

El archivo con las velocidades ofrecidas es actualizado, conservar la estructura del archivo, se tomó la columna mediana velocidad pero el archivo trae otras medidas.

In [12]:
df_velocidad = pd.read_csv('DATA_OOKLA/Nodos_vel_prom_hist1.csv', sep=';', decimal=',', 
                           usecols=['NODO','MEDIANA_VEL','ESTRATO','MES','ANIO'])
df_velocidad['FECHA'] = '20' + df_velocidad['ANIO'].astype('str') + '/' + df_velocidad['MES'].astype('str') + '/01'
df_velocidad['FECHA'] = df_velocidad['FECHA'].astype('datetime64[D]').astype('str')
df_velocidad = df_velocidad.groupby('NODO').agg({'MEDIANA_VEL':'median'}).reset_index()

In [13]:
df_group_dash = df_group.merge(df_velocidad[['NODO','MEDIANA_VEL']], on=['NODO'], how='left')
df_group_dash['down_mediana_velocidad'] = (df_group_dash['DOWNLOAD_KBPS'] /1000)/ df_group_dash['MEDIANA_VEL']
df_group_dash['OPER_MES_SEG'] = df_group_dash['OPERADOR'] + '_' + df_group_dash['MES_SEGMENTADO'].astype('str')
df_group_dash.to_csv('DATA_OOKLA/salidas/ookla_fechas_up_down.csv', sep=';', decimal=',', index=False)

# Tendecias Generales dash

In [14]:
t = time.time()
cols = ["FLAG_LAT","OPERADOR", "DOWNLOAD_KBPS","UPLOAD_KBPS", "NODO","FECHA"]
df_ookla_general = pd.read_csv('DATA_OOKLA/OOKLA_FIJA.csv', sep=',', usecols=cols)
mask_seg = (df_ookla_general['FLAG_LAT']==0)
df_ookla_general = df_ookla_general[mask_seg].copy()
df_ookla_general['FECHA'] = [str(x)[:7]+'-15' for x in df_ookla_general['FECHA']]
dic_funs = {"DOWNLOAD_KBPS":fun_tri_3, "UPLOAD_KBPS":fun_tri_3} 
df_ookla_general = df_ookla_general.groupby(["NODO","FECHA","OPERADOR"]).agg(dic_funs).reset_index()
df_ookla_general.to_csv('DATA_OOKLA/salidas/ookla_general.csv', sep=';', decimal=',', index=False)
print(f'Tiempo de ejecución en minutos: {(time.time()-t)/60}')
df_ookla_general.info()

Tiempo de ejecución en minutos: 3.9501426736513774
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238601 entries, 0 to 238600
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   NODO           238601 non-null  object 
 1   FECHA          238601 non-null  object 
 2   OPERADOR       238601 non-null  object 
 3   DOWNLOAD_KBPS  115236 non-null  float64
 4   UPLOAD_KBPS    115236 non-null  float64
dtypes: float64(2), object(3)
memory usage: 9.1+ MB
